# **Imports**
- Import necessary packages

In [ ]:
from __future__ import print_function 
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from torchvision import datasets, transforms 

# **Network Architecture**
- Define convolution blocks and the model structure

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), # Input - 28x28x1, Output - 28x28x16, RF - 3x3
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1), # Input - 28x28x16, Output - 28x28x16, RF - 5x5
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2) # Input - 28x28x16, Output - 14x14x16
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1), # Input - 14x14x16, Output - 14x14x16, RF - 10x10
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1), # Input - 14x14x16, Output - 14x14x16, RF - 12x12
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2) # Input - 14x14x16, Output - 7x7x16
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1), # Input - 7x7x16, Output - 7x7x16, RF - 24x24
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, 3, padding=1), # Input - 7x7x16, Output - 7x7x32, RF - 26x26
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 10, 3, padding=1), # Input - 7x7x32, Output - 7x7x10, RF - 28x28
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.AvgPool2d(7, 7) # Input - 7x7x10, Output - 1x1x10
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

# **Check Model Parameters**
- Testing dummy input with torchsummary

In [ ]:
!pip install torchsummary
from torchsummary import summary 
use_cuda = torch.cuda.is_available() 
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 16, 28, 28]           2,320
              ReLU-5           [-1, 16, 28, 28]               0
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,320
              ReLU-9           [-1, 16, 14, 14]               0
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,320
             ReLU-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
        MaxPool2d-14             [-1, 1

# **Dataset Preparation**
- Downloading dataset
- Normalising dataset
- Generator for train and test set

In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)) # 0.1307 is the mean, 0.3081 is the std. deviation of the entire dataset
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# **Define Train and Test functions**

In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    loss = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        pred = output.argmax(dim=1, keepdim=True)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        correct += pred.eq(target.view_as(pred)).sum().item()
      
    loss /= len(train_loader.dataset)

    # print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    #     loss, correct, len(train_loader.dataset),
    #     100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# **Run Model**

In [ ]:

model = Net().to(device)
lr = 0.01
for epoch in range(1, 20):
    if epoch > 8:
      lr = 0.001
    print('Epoch : ', epoch)
    optimizer = optim.SGD(model.parameters(), lr, momentum=0.9)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

Epoch :  1


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0832, Accuracy: 9841/10000 (98.41%)

Epoch :  2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0209, Accuracy: 9945/10000 (99.45%)

Epoch :  18


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0212, Accuracy: 9943/10000 (99.43%)

Epoch :  19


loss=0.012356108985841274 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.26it/s]



Test set: Average loss: 0.0207, Accuracy: 9945/10000 (99.45%)

